# Python optimization library

[DEAP documentation¶](https://deap.readthedocs.io/en/master/)

In [ ]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 1.1 MB/s eta 0:00:00


# Assume the following parameters used in ODE are concentration-indenpendent.

In [ ]:
import numpy as np
from scipy.integrate import odeint
from tqdm import tqdm
import time
from deap import base, creator, tools, algorithms
import random

# branch ratio

a21 = 1
a31 = 0.27
a32 = 0.73

a41 = 0.18
a42 = 0.24
a43 = 0.58

a51 = 0.24
a52 = 0.23
a53 = 0.2
a54 = 0.33

# decay rate of excited levels

W2 = 63000 # Tm, n2
W3 = 20000 # Tm, n3
W4 = 15000 # Tm, n4
W5 = 33000 # Tm, n5
Ws2 = 8000 # Yb, ns2

# Power density

P980 = 3*10**7


# define ODE


def system(state, t, c1, c2, c3, c4, k31, k41, k51):

    ns2, n1, n2, n3, n4, n5 = state

    #ns2
    ms2 = 1.23*P980*(54082-ns2) - Ws2*ns2 - (c1*n1+c2*n2+c3*n3+c4*n4)*ns2  # ns1 = total_Yb - ns2

    # n1
    m1 = -c1*n1*ns2 + a21*W2*n2 + a31*W3*n3 + a41*W4*n4 + a51*W5*n5 - k41*n1*n4 - k31*n1*n3 - k51*n5*n1

    # n2
    m2 = c1*n1*ns2 - c2*n2*ns2 - a21*W2*n2 + a32*W3*n3 + a42*W4*n4 + a52*W5*n5 + k41*n1*n4 + 2*k31*n1*n3

    # n3
    m3 = c2*n2*ns2 - c3*n3*ns2 - (a31+a32)*W3*n3 + a43*W4*n4 + a53*W5*n5 + 2*k51*n1*n5 + k41*n1*n4 - k31*n1*n3

    # n4
    m4 = c3*n3*ns2 - c4*n4*ns2 - (a43+a42+a41)*W4*n4 + a54*W5*n5 - k41*n1*n4

    # n5
    m5 = c4*n4*ns2 - (a54+a53+a52+a51)*W5*n5 - k51*n1*n5


    return [ms2, m1, m2, m3, m4, m5]



# Define the objective function for GA

In [ ]:
# objective function to minimize error, describe how well the ODE solution matches desired criteria

# update parameters with given Tm concentration and reference concentration Cr

Cr = 8 / 100


# update up-conversion parameters

def calculate_up(up, C2):
    C2 = C2 / 100
    return 3 * up / (2 + (Cr / C2) ** 2)

# update cross-relaxation parameters

def calculate_k(k, C2):
    C2 = C2 / 100
    return k * (C2 / Cr) ** 2

# calculate the NIR and blue emission at given concentration and parameters

def compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51):

    # update initial condition at each concentration
    y = 901.4 * x
    state0 = [0, y, 0, 0, 0, 0]

    # time range and step
    t = np.arange(0.0, 0.001, 0.000001)

    # solve the ODE
    state = odeint(system, state0, t, args=(c1, c2, c3, c4, k31, k41, k51))

    # NIR and blue emission
    NIR = a31 * W3 * state[:, 3][-1]
    blue_1 = a41 * W4 * state[:, 4][-1]
    blue_2 = a52 * W5 * state[:, 5][-1]
    blue_total = blue_1 + blue_2

    return NIR, blue_total


# define objective function for GA used later to evaluate each individual

def objective(exponents):

    # exponents of c1, c2, c3, c4, k31, k41, k51
    c1_exp, c2_exp, c3_exp, c4_exp, k31_exp, k41_exp, k51_exp = exponents

    # convert exponent values to actual values
    c1, c2, c3, c4, k31, k41, k51 = (10**c1_exp, 10**c2_exp, 10**c3_exp, 10**c4_exp, 10**k31_exp, 10**k41_exp, 10**k51_exp)

    # divide them into two groups, and later they will be updated with two different manners
    up_values = [c1, c2, c3, c4]
    k_values = [k31, k41, k51]


    # reference parameter Cr
    Cr = 8 / 100 # 8%

    # experimental data points: Tm concentration from 2%, 4%, 8%, 16%, Yb: 60%
    C2_range = [2,4,8,16]

    # storing new cross relaxation and new up-conversion values
    nested_up_list = []
    nested_k_list = []

    # for each concentration, update c1, c2, c3, c4, and update k31, k41, k51 respectively
    for C2 in C2_range:

        up_sublist = [calculate_up(up, C2) for up in up_values]

        k_sublist = [calculate_k(k, C2) for k in k_values]

        nested_up_list.append(up_sublist)

        nested_k_list.append(k_sublist)



    # Iterate over x values and compute NIR and blue_total for each

    x_values = [2,4,8,16]

    exp_NIR = [0.6*10**4, 2*10**4, 5.2*10**4, 7.4*10**4]

    exp_blue_total = [1.3*10**4, 1.5*10**4, 2.3*10**4, 1.4*10**4]


    total_error = 0

    # calculate NIR and blue values and accumulate the error for 4 data points (four Tm concentrations: 2%, 4%, 8%, 16%)

    for index, x in enumerate(tqdm(x_values)):

        c1, c2, c3, c4 = nested_up_list[index]

        k31, k41, k51 = nested_k_list[index]

        NIR, blue_total = compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51)

        error_NIR = (NIR - exp_NIR[index])**2
        error_blue = (blue_total - exp_blue_total[index])**2
        total_error += np.sqrt(error_NIR + error_blue)


    return total_error, # a return statement with a trailing comma, it means the function is returning a tuple with a single element.



# GA (this code snippet costs 10 min)

(doesn't necessarily give the desired parameters each time)

In [ ]:
# Define the following eight parameters for genetic algorithm


CXPB = 0.7 # the probability to mix two individuals

MUTPB = 0.2 # the probability that one individual mutate or not

alpha1 = 0.1 # mix two individuals together(like the meaning of weight)

Mut_p = 0.2 # the probability that one component of one individual mutate or not

Average = 0 # use the Normal distributation(average=0, std=3) to generate a random number to change the component of one individual
Std = 3


POP_SIZE = 100 # population size
NGEN = 60 # how many generations (iteration for 60 times)



Cr = 8 / 100

# define fitness: aim to minimize its values
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# define the class of each individual
creator.create("Individual", list, fitness=creator.FitnessMin)

# The toolbox is a container for various functions used in the genetic algorithm
toolbox = base.Toolbox()


# create the object from the class defined before
valid_values = [i for i in range(-10, 10)]
toolbox.register("attr_float", random.choice, valid_values)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=7)


# define the class of each population
toolbox.register("population", tools.initRepeat, list, toolbox.individual) # create a list of individuals, essentially generating the initial population

# Genetic Operators: crossover
toolbox.register("mate", tools.cxBlend, alpha= alpha1)

# Genetic Operators: mutate
toolbox.register("mutate", tools.mutGaussian, mu=Average, sigma=Std, indpb=Mut_p)


# Evaluation Function based on the objective function defined before
toolbox.register("evaluate", objective)

# select the best one from every three candidates
toolbox.register("select", tools.selTournament, tournsize=3)



# initialize the first generation with 100 individuals
pop = toolbox.population(n=POP_SIZE)

# evaluate the fitness of each individual
fits = list(map(toolbox.evaluate, pop))
for fit, ind in zip(fits, pop):
    ind.fitness.values = fit


# runing the GA with 60 generations
for gen in range(NGEN):

    # create the offspring
    offspring = algorithms.varAnd(pop, toolbox, CXPB, MUTPB)

    # evaluate the fitness of each individual in the created offspring
    for ind in offspring:
        fit = toolbox.evaluate(ind)
        ind.fitness.values = fit

    # select 100 individuals from offsprings as the next generation
    pop = toolbox.select(offspring, k=len(pop))



# output the best ten individuals from the last generations(60th)
top10 = tools.selBest(pop, k=10)



Streaming output truncated to the last 5000 lines.
100%|██████████| 4/4 [00:00<00:00, 66.97it/s]


In [52]:
# top 10 individuals
for ind in top10:
    print(ind, ind.fitness.values)

[3.5481529473960123, -3.529519903736616, -1.1164402736064272, -1.9762826699140286, -0.3499641257960679, -3.36549401431438, -3.8195592141344146] (63151.46989200831,)
[3.5481529473960123, -3.529519903736616, -1.1164402736064272, -1.9762826699140286, -0.3499641257960679, -3.36549401431438, -3.8195592141344146] (63151.46989200831,)
[3.5482576196628894, -3.5295199533008823, -1.1164401236827188, -1.976284717367502, -0.3499633455216188, -3.544794432382538, -3.8672083399577253] (63151.470946524445,)
[3.5482576196628894, -3.5295199533008823, -1.1164401236827188, -1.976284717367502, -0.3499633455216188, -3.544794432382538, -3.8672083399577253] (63151.470946524445,)
[3.5482764888640066, -3.529519946937249, -1.1164400678804034, -1.9762853640834668, -0.3499108179156852, -3.568040065239673, -3.869055895433715] (63151.471257284626,)
[3.5482764888640066, -3.529519946937249, -1.1164400678804034, -1.9762853640834668, -0.3499108179156852, -3.568040065239673, -3.869055895433715] (63151.471257284626,)
[3.5

In [53]:
import numpy as np

# Extracting the exponent values
c1_exp, c2_exp, c3_exp, c4_exp, k31_exp, k41_exp, k51_exp = top10[0]

# Converting exponent values to actual values
c1, c2, c3, c4, k31, k41, k51 = (10**c1_exp, 10**c2_exp, 10**c3_exp,
                                 10**c4_exp, 10**k31_exp, 10**k41_exp, 10**k51_exp)

up_values = [c1, c2, c3, c4]

k_values = [k31, k41, k51]

# reference parameter Cr
Cr = 8 / 100  # 8%

# update up-conversion

def calculate_up(up, C2):
    C2 = C2 / 100
    return 3 * up / (2 + (Cr / C2) ** 2)

# update cross-relaxation

def calculate_k(k, C2):
    C2 = C2 / 100
    return k * (C2 / Cr) ** 2


# Tm concentration from 2% to 40%
C2_range = np.arange(2, 41, 1)

# storing new cross relaxation and new up-conversion values
nested_up_list = []
nested_k_list = []


for C2 in C2_range:

    up_sublist = [calculate_up(up, C2) for up in up_values]

    k_sublist = [calculate_k(k, C2) for k in k_values]

    nested_up_list.append(up_sublist)

    nested_k_list.append(k_sublist)

# nested_k2_list now contains the K2 values as required


In [54]:
from scipy.integrate import odeint
from tqdm import tqdm

def compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51):

    # initial condition
    y = 901.4 * x
    state0 = [0, y, 0, 0, 0, 0]

    t = np.arange(0.0, 0.001, 0.000001)

    state = odeint(system, state0, t, args=(c1, c2, c3, c4, k31, k41, k51))

    # Compute NIR and blue
    NIR = a31 * W3 * state[:, 3][-1]
    blue_1 = a41 * W4 * state[:, 4][-1]
    blue_2 = a52 * W5 * state[:, 5][-1]
    blue_total = blue_1 + blue_2

    return NIR, blue_total

# Iterate over x values and compute NIR and blue emission intensity

x_values = list(np.arange(2, 41, 1))
NIR_values = []
blue_total_values = []


for index, x in enumerate(tqdm(x_values)):

    c1, c2, c3, c4 = nested_up_list[index]

    k31, k41, k51 = nested_k_list[index]

    NIR, blue_total = compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51)

    # Store the results
    NIR_values.append(NIR)
    blue_total_values.append(blue_total)


100%|██████████| 39/39 [00:00<00:00, 55.41it/s]


In [55]:
import plotly.graph_objects as go
from IPython.core.display import display, HTML

# Create a figure
fig1 = go.Figure()

fig1.add_trace(go.Scatter(x=x_values, y=NIR_values, mode='lines+markers', name='NIR simulation', marker=dict(color='orange')))
fig1.add_trace(go.Scatter(x=x_values, y=blue_total_values, mode='lines+markers', name='Blue simulation', marker=dict(color='blue')))


y_ticks = [i for i in range(int(min(min(NIR_values), min(blue_total_values)) / 1e4),
                            int(max(max(NIR_values), max(blue_total_values)) / 1e4) + 1)]

fig1.update_yaxes(tickvals=[tick * 1e4 for tick in y_ticks],
                 ticktext=[f"{tick} × 10^4" for tick in y_ticks])


fig1.update_xaxes(title_font=dict(size=18, color='black'),tickfont=dict(size=14),showgrid=True, gridwidth=2, gridcolor='white', dtick=2)
fig1.update_yaxes(title_font=dict(size=18, color='black'),tickfont=dict(size=14),showgrid=True, gridwidth=2, gridcolor='white')


fig1.update_layout(
    title_text='NaYF4: 60% Yb, x% Tm',
    title_font=dict(size=24, family="Arial, sans-serif", color='royalblue'),
    width=600, height=700,
    xaxis_title='Tm concentration(%)',
    yaxis_title='Intensity(pps)',
    title_x=0.42,  # center the main title
    title_y=0.95,  # adjust the vertical position of the main title
    showlegend=True, margin=dict(l=20, r=20, b=20, t=70),
    legend=dict(
        font=dict(
            size=20,
            family="Arial, sans-serif"
        )
    )
    )



# Here are the better parameters generated before

In [61]:
a1=[3.3285842685555322, -3.5369813062456514, -0.5938554760649145, -1.7116852065474428, -3.4031198011311172, 3.1548047577113, 4.570284374433792]
a2=[3.3285842685555322, -3.5369813062456514, -0.5938554760649145, -1.7116852065474428, -3.4031198011311172, 3.1548047577113, 4.570284374433792]
a3=[3.3285842685555322, -3.5369813062456514, -0.5938554760649145, -1.7116852065474428, -3.4031198011311172, 3.1548047577113, 4.570284374433792]
a4=[3.3285842685555322, -3.5369813062456514, -0.5938554760649145, -1.7116852065474428, -3.4031198011311172, 3.1548047577113, 4.570284374433792]
a5=[3.3285842685555322, -3.5369813062456514, -0.5938554760649145, -1.7116852065474428, -3.4031198011311172, 3.1548047577113, 4.570284374433792]
a6=[3.3285842685555322, -3.5369813062456514, -0.5938554760649145, -1.7116852065474428, -3.4031198011311172, 3.1548047577113, 4.570284374433792]
a7=[3.3275993034983853, -3.536981054975469, -0.5966175851045443, -1.9671504731508331, -3.5192804002203224, 3.1546061288653346, 7.265011573365139]
a8=[3.325276090890399, -3.5369817075033456, -0.5964628022319227, -1.906967861564128, -2.802021565249875, 3.154512026058176, 4.6686937029462845]
a9=[3.325276090890399, -3.5369817075033456, -0.5964628022319227, -1.906967861564128, -2.802021565249875, 3.154512026058176, 4.6686937029462845]
a10=[3.314027516107214, -3.536981070725067, -0.5911785844014255, -1.8496031109172808, -1.4000015042353269, 3.1549837080108905, 5.116632475711274]
aa=[a1,a2,a3,a4,a5,a6,a7,a8,a9,a10]

---
# The parameters distribution


In [62]:
import plotly.graph_objects as go
from IPython.core.display import display, HTML


# Extract the top ten individuals from each run (there are totally NUM_RUNS=10 runs)



for i in range(1):

    top_ten_individuals = aa

    # Unzipping the parameters
    c1_exps, c2_exps, c3_exps, c4_exps, k31_exps, k41_exps, k51_exps = zip(*top_ten_individuals)

    # Create subplots
    fig = go.Figure()

    # Adding subplots for each parameter
    params = [c1_exps, c2_exps, c3_exps, c4_exps, k31_exps, k41_exps, k51_exps]
    param_names = ["c1_exp", "c2_exp", "c3_exp", "c4_exp", "k31_exp", "k41_exp", "k51_exp"]

    for param, name in zip(params, param_names):
        fig.add_trace(go.Box(
            y=param,
            name=name
        ))


    fig.update_xaxes(showgrid=True, gridwidth=2, gridcolor='white')
    fig.update_yaxes(showgrid=True, gridwidth=2, gridcolor='white')

    fig.update_layout(
                title=f"Parameter exponents of top ten individuals",
                title_font=dict(size=24, family="Courier New, monospace"),
                title_x=0.5,  # center
                title_y=0.9,
                width=800, height=700,
                yaxis_title="Parameter Exponent Value",
                xaxis_title="Seven Parameter Exponents",
                xaxis=dict(
                title_font=dict(size=22, family="Courier New, monospace"),
                tickfont=dict(size=22, family="Courier New, monospace")
            ),
            yaxis=dict(
                title_font=dict(size=22, family="Courier New, monospace"),
                tickfont=dict(size=22, family="Courier New, monospace")
            )
            )



    fig.show()

In [58]:

# transpose_matrix = list(map(list, zip(*aa)))

# averages=[]
# summation=0

# for i in range(7):
#     for j in range(10):
#         summation += transpose_matrix[i][j]
#     averages.append(summation/10)


In [63]:
# averages = [sum(x)/len(x) for x in zip(*aa)] (TypeError: 'list' object is not callable)


# Unpack the averages to respective variables
c1_exp_avg, c2_exp_avg, c3_exp_avg, c4_exp_avg, k31_exp_avg, k41_exp_avg, k51_exp_avg = aa[0]

c1, c2, c3, c4, k31, k41, k51 = 10**c1_exp_avg, 10**c2_exp_avg, 10**c3_exp_avg, 10**c4_exp_avg, 10**k31_exp_avg, 10**k41_exp_avg, 10**k51_exp_avg


up_values = [c1, c2, c3, c4]

k_values = [k31, k41, k51]

# reference parameter Cr
Cr = 8 / 100  # 8%

# update up-conversion

def calculate_up(up, C2):
    C2 = C2 / 100
    return 3 * up / (2 + (Cr / C2) ** 2)

# update cross-relaxation

def calculate_k(k, C2):
    C2 = C2 / 100
    return k * (C2 / Cr) ** 2


# Tm concentration from 2% to 40%
C2_range = np.arange(2, 41, 1)

# storing new cross relaxation and new up-conversion values
nested_up_list = []
nested_k_list = []


for C2 in C2_range:

    up_sublist = [calculate_up(up, C2) for up in up_values]

    k_sublist = [calculate_k(k, C2) for k in k_values]

    nested_up_list.append(up_sublist)

    nested_k_list.append(k_sublist)

# nested_k2_list now contains the K2 values as required

from scipy.integrate import odeint
from tqdm import tqdm

def compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51):

    # Compute initial condition
    y = 901.4 * x
    state0 = [0, y, 0, 0, 0, 0]

    # ODEs
    t = np.arange(0.0, 0.001, 0.000001)

    state = odeint(system, state0, t, args=(c1, c2, c3, c4, k31, k41, k51))

    # Compute NIR and blue_total
    NIR = a31 * W3 * state[:, 3][-1]
    blue_1 = a41 * W4 * state[:, 4][-1]
    blue_2 = a52 * W5 * state[:, 5][-1]
    blue_total = blue_1 + blue_2

    return NIR, blue_total

# Iterate over x values and compute NIR and blue_total for each
x_values = list(np.arange(2, 41, 1))  # Range of C2 values from 2% to 40%
NIR_values = []
blue_total_values = []


for index, x in enumerate(tqdm(x_values)):

    c1, c2, c3, c4 = nested_up_list[index]

    k31, k41, k51 = nested_k_list[index]

    NIR, blue_total = compute_values_for_x(x, c1, c2, c3, c4, k31, k41, k51)

    # Store the results
    NIR_values.append(NIR)
    blue_total_values.append(blue_total)

sum=[i+j for i,j in zip(NIR_values, blue_total_values)]


# You can now use NIR_values, blue_total_values, ratios1, etc., for further analysis

import plotly.graph_objects as go
from IPython.core.display import display, HTML

# Create a figure
fig1 = go.Figure()

fig1.add_trace(go.Scatter(x=x_values, y=NIR_values, mode='lines+markers', name='NIR simulation', marker=dict(color='orange')))
fig1.add_trace(go.Scatter(x=x_values, y=blue_total_values, mode='lines+markers', name='Blue simulation', marker=dict(color='blue')))
#fig1.add_trace(go.Scatter(x=x_values, y=sum, mode='lines+markers', name='total simulation', marker=dict(color='black')))


# Update y-axis to show values in units of 10^4
# Calculate the range for y-axis ticks based on your data's range


y_ticks = [i for i in range(int(min(min(NIR_values), min(blue_total_values)) / 1e4),
                            int(max(max(NIR_values), max(blue_total_values)) / 1e4) + 1)]

fig1.update_yaxes(tickvals=[tick * 1e4 for tick in y_ticks],
                 ticktext=[f"{tick} × 10^4" for tick in y_ticks])


fig1.update_xaxes(title_font=dict(size=18, color='black'),tickfont=dict(size=14),showgrid=True, gridwidth=2, gridcolor='white', dtick=2)
fig1.update_yaxes(title_font=dict(size=18, color='black'),tickfont=dict(size=14),showgrid=True, gridwidth=2, gridcolor='white')


fig1.update_layout(
    title_text='NaYF4: 60% Yb, x% Tm',  # Main Title
    title_font=dict(size=24, family="Arial, sans-serif", color='royalblue'),
    width=600, height=700,
    xaxis_title='Tm concentration(%)',
    yaxis_title='Intensity(pps)',
    title_x=0.42,  # Center the main title
    title_y=0.95,  # Adjust the vertical position of the main title
    showlegend=True, margin=dict(l=20, r=20, b=20, t=70),
    legend=dict(
        font=dict(
            size=20,
            family="Arial, sans-serif"
        )
    )
    )




100%|██████████| 39/39 [00:00<00:00, 92.62it/s]
